# Calculating Depths of Catalogs and Images

This notebook can be run to regenerate the figures in [DMTN-296](). It uses simulated data from Operations Rehearsal 4 (OR4) and (optionally) HSC PDR2 rerun.

In [ ]:
%matplotlib inline
import numpy as np
import pylab as plt
import pandas as pd
from tqdm import tqdm
import astropy.units as u

In [ ]:
from lsst.daf.butler import Butler
import lsst.daf.base as dafBase

In [ ]:
BAND_COLORS = dict([
    ('u','#56b4e9'),
    ('g','#008060'),
    ('r','#ff4000'),
    ('i','#850000'),
    ('z','#6600cc'),
    ('Y','#000000'),
])

In [ ]:
# OR4 LSSTComCamSim
dataset_name='or4'
repo='embargo_or4'
collection='LSSTComCamSim/runs/DRP/OR4/w_2024_25/DM-45066'
butler = Butler(repo, collections=collection)
dataId = {'visit': 7024062600787, 'instrument': 'LSSTComCamSim', 'detector': 4}

## Catalog Depth Estimates

The following code block estimates depths from the single-visit source catalog data. We derive the magnitude limit in two ways:
1. Taking the mean magnitude of sources with 4.75 < SNR < 5.25
2. Performing a linear fit to log10(magerr) vs mag and solving for mag where magerr = 0.2171

In [ ]:
calib = butler.get('calexp.photoCalib', dataId=dataId)
#calib = calexp.getPhotoCalib()
#src = butler.get('sourceTable', dataId=dataId)
src = butler.get('src', dataId=dataId)
table = src.asAstropy()
srcTable = butler.get('sourceTable', dataId=dataId)
assert len(table) == len(srcTable)
assert np.all(table['id'] == srcTable.index)
print(f"Number of sources: {len(table)}")

Some notes about single-frame photometry...
1. The `psfFlux` in the SourceTable has had local calibrations applied (it is not an instrumental flux). The application of the local calibration is (probably) done in the `TransformSourceTableTask` using configs [here](https://github.com/lsst/pipe_tasks/blob/79a68fa90d8ec248848db68ccb162ba09b926475/schemas/Source.yaml#L244-L257) and code in [LocalNanojansky](https://github.com/lsst/pipe_tasks/blob/79a68fa90d8ec248848db68ccb162ba09b926475/python/lsst/pipe/tasks/functors.py#L1751).
2. The calibrations are (slightly) different between the SourceCatalog (`base_LocalPhotoCalib`) and the SourceTable (`localPhotoCalib`). This may not be surprising if the calibrations are run multiple times.
3. It is possible to recover the `psfFlux` using the `localPhotoCalib`. This can be formally or informally. See below for an example.

In [ ]:
# Convert flux to magnitude
flux = srcTable['psfFlux']
fluxerr = srcTable['psfFluxErr']

# Convert from nJy to AB mag
with np.errstate(invalid='ignore'):
    nJytoAB = (1*u.nJy).to(u.ABmag).value
    mag = -2.5*np.log10(flux) + nJytoAB
    magerr = 2.5/np.log(10) * fluxerr / flux

snr = flux/fluxerr
srcTable['psfMag'] = mag
srcTable['psfMagErr'] = magerr
srcTable['psfFluxSnr'] = snr

In [ ]:
# Select stars using extendedness (follows analysis_tools)
#extendedness = table['base_ClassificationExtendedness_value']
extendedness = srcTable['extendedness']
star = (extendedness >= 0) & (extendedness < 0.5)

In [ ]:
# Perform the catalog determination
snrval = 5.0
snrmin, snrmax = 4.75, 5.25
print(f"SNR selection: {snrmin} < SNR < {snrmax}")
snrsel = (snr > snrmin) & (snr < snrmax)
maglim_snr = np.mean(mag[star & snrsel])
mag_min, mag_max = np.min(mag[star & snrsel]), np.max(mag[star & snrsel])
print(f"Magnitude Range: {mag_max:.2f} >= mag >= {mag_min:.2f}")

In [ ]:
# Fit magerr vs mag
magerrval = 2.5/np.log(10) * (1/snrval)
print(f"magerr value: {magerrval:.4f}")
errsel = (magerr > magerrval/2.0) & (magerr < 2.0*magerrval)
poly = np.polynomial.Polynomial.fit(mag[star & errsel], np.log10(magerr[star & errsel]) - np.log10(magerrval), deg=1)
maglim_err = poly.roots()[0]
fn = lambda x: 10**(poly(x) + np.log10(magerrval))

In [ ]:
fig, ax = plt.subplots(1,2, figsize=(12, 5), sharex=True)
ax[0].set_xlim(21,27)

kw = dict(s=3, rasterized=True)

# Plot the snr selection
ax[0].axhline(snrval, ls='--', color='gray', alpha=0.5)
ax[0].fill_between(ax[0].get_xlim(),snrmin, snrmax, color='r', alpha=0.3)
ax[0].axvline(maglim_snr, ls='--', color='r', alpha=0.5, label=f"maglim={maglim_snr:.2f}")
ax[0].scatter(mag, snr, color='0.5', **kw)
ax[0].scatter(mag[star], snr[star], color='k', **kw)
ax[0].scatter(mag[star & snrsel], snr[star & snrsel], color='r', **kw)
ax[0].set_ylim(-0.1, 15)
ax[0].set_ylabel("PSF Flux SNR")
ax[0].set_xlabel("psfMag")
ax[0].legend()

# Plot the magerr fit
ax[1].axhline(magerrval, ls='--', color='gray', alpha=0.5)
ax[1].axvline(maglim_err, ls='--', color='r', alpha=0.5, label=f"maglim={maglim_err:.2f}")
ax[1].scatter(mag, magerr, color='0.5', **kw)
ax[1].scatter(mag[star], magerr[star], color='k', **kw)
ax[1].scatter(mag[star & errsel], magerr[star & errsel], color='r', **kw)
ax[1].plot(ax[1].get_xlim(), fn(np.array(ax[1].get_xlim())), ls=':', color='b')
ax[1].set_yscale('log')
ax[1].set_ylim(0.01, 1.0)
ax[1].set_ylabel("psfMagErr")
ax[1].set_xlabel("psfMag")
ax[1].legend()

fig.suptitle(collection+'\n'+str(dataId))

outfile=f'{dataset_name}_maglim_catalog.pdf'
plt.savefig(outfile)

## Summary Statistics Depth Estimate

The following code estimates the depth from summary statistics.

In [ ]:
def compute_magnitude_limit(
        psfArea,
        skyBg,
        zeroPoint,
        readNoise=0,
        gain=1.0,
        snr=5
):
    """Compute the expected point-source magnitude limit at a given                                                                               
    signal-to-noise ratio given the exposure-level metadata. Based on                                                                             
    the signal-to-noise formula provided in SMTN-002 (see LSE-40 for                                                                              
    more details on the calculation).                                                                                                             
                                                                                                                                                  
      SNR = C / sqrt( C/g + (B/g + sigma_inst**2) * neff )                                                                                          
                                                                                                                                                  
    where C is the counts from the source, B is counts from the (sky) 
    background, sigma_inst is the instrumental (read) noise, neff is 
    the effective size of the PSF, and g is the gain in e-/ADU.
    Note that input values of skyBg, zeroPoint, and readNoise should 
    all consistently be in electrons or ADU.
                                                                                                                                                  
    Parameters                                                                                                                                    
    ----------                                                                                                                                    
    psfArea  : `float`                                                                                                                            
        The effective area of the PSF [pix].                                                                                                      
    skyBg     : `float`                                                                                                                           
        The sky background counts for the exposure [ADU or e-].                                                                                   
    zeroPoint : `float`                                                                                                                           
        The zeropoint (includes exposure time) [ADU or e-].                                                                                       
    readNoise : `float`, optional                                                                                                                 
        The instrumental read noise for the exposure [ADU or e-].                                                                                        
    gain      : `float`, optional                                                                                                                 
        The instrumental gain for the exposure [e-/ADU]. The gain should                                                                          
         be set to 1 if the skyBg and zeroPoint are in units of e-.                                                                               
    snr       : `float`, optional                                                                                                                 
        Signal-to-noise ratio at which magnitude limit is calculated.                                                                             
                                                                                                                                                  
    Returns                                                                                                                                       
    -------                                                                                                                                       
    magnitude_limit : `float`                                                                                                                     
        The expected magnitude limit at the given signal to noise.                                                                                
    """
    # Effective number of pixels within the PSF calculated directly                                                                               
    # from the PSF model.                                                                                                                         
    neff = psfArea

    # Instrumental noise (read noise only)                                                                                                        
    sigma_inst = readNoise

    # Total background counts derived from Eq. 45 of LSE-40                                                                                       
    background = (skyBg/gain + sigma_inst**2) * neff
    # Source counts to achieve the desired SNR (from quadratic formula)                                                                           
    source = (snr**2)/(2*gain) + np.sqrt((snr**4)/(4*gain) + snr**2 * background)

    # Convert to a magnitude using the zeropoint.                                                                                                 
    # Note: Zeropoint currently includes exposure time                                                                                            
    magnitude_limit = -2.5*np.log10(source) + zeroPoint

    return magnitude_limit

In [ ]:
def plot_comparison(xvar, yvar, band, xlabel=None, ylabel=None, title=None, fraction=True):
    """ Plot comparison of two values.

    Parameters
    ----------
    xvar : the x-values
    yvar : the y-values
    band : the band
    xlabel : x-axis label
    ylabel : y-axis label
    title  : figure title
    fraction : plot fractional residuals
    """
    fig, ax = plt.subplots(1,2, figsize=(12,5))
    
    plt.sca(ax[0])
    kwargs = dict(s=2, rasterized=True)
    for b in ['g', 'r', 'i']:
        sel = band == b
        kwargs.update(color = BAND_COLORS[b], label=b)
        plt.scatter(xvar[sel], yvar[sel], **kwargs)
    plt.legend()
    plt.plot(plt.gca().get_xlim(), plt.gca().get_xlim(), ls='--', color='gray')
    if xlabel: plt.xlabel(xlabel) 
    if ylabel: plt.ylabel(ylabel)
    
    plt.sca(ax[1])
    if fraction:
        residual = (yvar - xvar)/xvar
    else:
        residual = (yvar - xvar)
    mean, sigma = np.mean(residual), np.std(residual)
    bins = np.linspace(*np.percentile(residual, [0.5, 99.5]))
    kwargs = dict(bins=bins, histtype='step', rasterized=True)
    for b in ['g', 'r', 'i']:
        sel = band == b
        kwargs.update(color=BAND_COLORS[b], label=b)
        plt.hist(residual[sel], **kwargs)
    plt.axvline(0, ls='--', color='gray')
    lines = plt.plot(np.nan, np.nan, label=f"$\mu = {mean:.2f} \pm {sigma:.2f}$")
    plt.legend(handles=lines,handlelength=0)

    if xlabel and ylabel:
        if fraction:
            plt.xlabel(f'({ylabel} - {xlabel})/{xlabel}')
        else:
            plt.xlabel(f'{ylabel} - {xlabel}')

    if title:    
        plt.suptitle(title)

In [ ]:
# OR4 LSSTComCamSim
dataset_name='or4'
repo='embargo_or4'
collection='u/kadrlica/m5_metrics_detector4'
butler = Butler(repo, collections=collection)

instrument = 'LSSTComCamSim'
detector=4

In [ ]:
# Get the ccdVisitTable
ccdVisit = butler.get('ccdVisitTable', dataId={'instrument': instrument})
ccdVisit = ccdVisit.loc[ccdVisit['detector'] == detector]
print(f"Found ccdVisit summary stats for {len(ccdVisit)} visits")

In [ ]:
# Unfortunately, the psfArea isn't passed out to ccdVisit so we need to get it.

# This can take a ~5 minutes, so save/load the output
filename = 'summary_stats.csv'
if os.path.exists(filename):
    print(f"Loading summary stats from {filename}.")
    summaryStats = pd.read_csv(filename)
else:
    print(f"Getting psfArea from summary stats...")
    summaryStats = ccdVisit.copy()
    psfArea = []
    for idx, row in tqdm(ccdVisit.iterrows(), total=len(ccdVisit)):
        dataId = {'instrument': instrument, 'visit': row.visitId, 'detector': row.detector}
        stats = butler.get('calexp.summaryStats', dataId=dataId)
        psfArea.append(stats.psfArea)
        
    summaryStats['psfArea'] = psfArea
    print(f"Writing summary stats to {filename}.")
    summaryStats.to_csv(filename)

print(f"Found summary stats for {len(summaryStats)}")
summaryStats

In [ ]:
# This can take about a minute, so save/load the output

filename = 'm5_metrics.csv'
if os.path.exists(filename):
    print(f"Loading summary stats from {filename}.")
    m5 = pd.read_csv(filename)
else:
    print(f"Getting m5 metrics...")

    dtype="sourceTable_visitFiveSigmaDepth_metrics"
    refs = sorted(butler.registry.queryDatasets(dtype))

    m5 = []
    for ref in tqdm(refs, total=len(refs)):
        out = dict(visit=ref.dataId['visit'],
                   band=ref.dataId['band'],
                   day_obs=ref.dataId['day_obs'])
    
        metrics = butler.get(ref)['m5Metric']
        for metric in metrics:
            out[str(metric.metric_name)] = metric.quantity.value
    
        m5.append(out)
    
    m5 = pd.DataFrame(m5)
    print(f"Writing m5 metrics to {filename}.")
    m5.to_csv(filename)

print(f"Found m5 metrics for {len(m5)} visits")

In [ ]:
# Merge the summary stats with the m5 metrics
df = pd.merge(m5[['visit','mean5sigmaDepth','median5sigmaDepth']], summaryStats, 
              left_on='visit', right_on='visitId', how='inner')
df

In [ ]:
# Get the gain and read noise (same for all visits)
dataId = {'visit': 7024062600787, 'instrument': 'LSSTComCamSim', 'detector': 4}
metadata = butler.get('calexp.metadata',dataId=dataId)

readNoise = np.mean([metadata[k] for k in metadata.keys() if k.startswith('LSST READNOISE')])
print(f"readNoise: {readNoise:.2f} [e-]")

gain = np.mean([metadata[k] for k in metadata.keys() if k.startswith('LSST GAIN')])
print(f"gain: {gain:.2f} [ADU/e-]")

units = metadata['LSST ISR UNITS']
print(f"image units: [{units}]")

PIXSCALE = 0.2
print(f"pixel scale: {PIXSCALE} arcsec/pix")

In [ ]:
# Define the input summary statistics.

# Much better to have the psfArea
psfArea = df['psfArea']

# Sky background currently in ADU
skyBgAdu = df['skyBg'] # ADU
skyBgElec = skyBgAdu * gain # e-

# Zeropoint currently in ADU
zeroPointAdu = df['zeroPoint'] #ADU (includes expTime)
zeroPointElec = zeroPointAdu + 2.5*np.log10(gain) # e- (includes expTime)

# Read noise currently in e-
readNoiseElec = readNoise # e-
readNoiseAdu = readNoise / gain # ADU

In [ ]:
# Check that the maglim calculated in ADU and electrons are the same.

# Compute the magnitude limit in ADU
maglim_adu = compute_magnitude_limit(psfArea, skyBgAdu, zeroPointAdu,
                                     readNoise=readNoiseAdu, gain=gain)
# Compute the magnitude limit in electrons
maglim_elec = compute_magnitude_limit(psfArea, skyBgElec, zeroPointElec,
                                          readNoise=readNoiseElec, gain=1.0)
assert np.allclose(maglim_adu, maglim_elec, equal_nan=True)

In [ ]:
# Calculate the summary stats maglim.

# The OR4 variance planes were made (incorrectly) using the image variance in [ADU] and 
# the readNoise in [e-]. This propagates to the catalog errors, so making this switch 
# leads to better agreement in the depths.

# OR4 units
#img_units='ADU'; rn_units='e-'

# Consistent units
img_units='ADU'; rn_units='ADU'

print("Calculating magnitude limit...")
print(f"Image units: {img_units}")
print(f"readNoise units: {rn_units}")      
df['magLim5'] = compute_magnitude_limit(psfArea, 
                                        skyBgElec if img_units=='e-' else skyBgAdu, 
                                        zeroPointElec if img_units=='e-' else zeroPointAdu,
                                        readNoise=readNoiseElec if rn_units=='e-' else readNoiseAdu, 
                                        gain=1.0 if img_units=='e-' else gain)

In [ ]:
plot_comparison(df['magLim5'], df['mean5sigmaDepth'], band=df['band'],
                xlabel='magLim5 [stats]', ylabel='mean5sigmaDepth [cat]',
                fraction=False)
plt.savefig(f'{dataset_name}_maglim_compare_{img_units.lower()}_{rn_units.lower()}.pdf')

# Comparison to Predicted Depth from OpSim 

OpSim cacluates a predicted SNR=5 depth for point sources following the prescription described in [SMTN-002](https://smtn-002.lsst.io/). Here we compare the OpSim prediction to the output summary stats from the Science Pipelines. We specifically investigate the seeing, zero point, sky background, and limiting magnitude.

In [ ]:
# Load the opsim data matched to nightlyValidation
dataset_name="or4_opsim"
opsim = pd.read_csv('or4_opsim_match.csv')
columns = ['visit', 'seeingFwhmEff', 'seeingFwhmGeom', 'skycounts', 'zeropoint', 'fiveSigmaDepth']
opsim = pd.merge(opsim[columns], df, on='visit', how='inner')
opsim.reset_index(inplace=True)

In [ ]:
# PSF Area
psfArea_opsim = 2.266*(opsim['seeingFwhmEff']/PIXSCALE)**2
plot_comparison(opsim['psfArea'], psfArea_opsim, opsim['band'], 
                xlabel='psfArea [DM] (pix)', ylabel='psfArea [opsim] (pix)')
plt.savefig(f'{dataset_name}_psf.pdf')

In [ ]:
# Zeropoint
zeroPoint_opsim = opsim['zeropoint']+2.5*np.log10(30/gain)
plot_comparison(opsim['zeroPoint'], zeroPoint_opsim, opsim['band'], 
                xlabel='zeroPoint [DM]', ylabel='zeropoint [opsim]',
               fraction=False)
plt.savefig(f'{dataset_name}_zeropoint.pdf')

In [ ]:
# Sky background
skyBg_opsim = opsim['skycounts']/gain # ADU
plot_comparison(opsim['skyBg'], skyBg_opsim, opsim['band'], 
                xlabel='skyBg [DM] (ADU)', ylabel='skyBg [opsim] (ADU)')
plt.savefig(f'{dataset_name}_sky.pdf')

In [ ]:
# Magnitude Limit
magLim5_opsim = opsim['fiveSigmaDepth'] 
plot_comparison(opsim['magLim5'], magLim5_opsim, opsim['band'], 
                xlabel='magLim5 [DM]', ylabel='fiveSigmaDepth [opsim]',
                fraction=False)
plt.savefig(f'{dataset_name}_m5.pdf')

## Additional PSF Comparisons

In addition to the primary comparisons performed above, it seemed useful to do a few more comparisons of PSF-related quantities. In particular, the relationship between the `psfSigma` and `psfArea` is useful to develop. Note that `psfArea` is the measured quantity and `psfSigma` is derived. Specifically, the `psfSigma` is the determinent radius of the PSF ([shape.getDeterminantRadius()](https://github.com/lsst/pipe_tasks/blob/da64303b90586b287562c70ddaad676401150d60/python/lsst/pipe/tasks/computeExposureSummaryStats.py#L332)). The fact that the DM `psfArea` can match OpSim `neff`, while the DM `psfSigma` does not match OpSim `seeingFwhmEff` or `seeingFwhmGeom` is telling us something about the difference in the shape of the PSF in OR4 measured by DM vs the OpSim assumptions based on some older zemax modeling.

In [ ]:
psfSigmaGeom_opsim = opsim['seeingFwhmGeom']/2.355/PIXSCALE
plot_comparison(opsim['psfSigma'], psfSigmaGeom_opsim, opsim['band'], 
                xlabel='psfSigma', ylabel='seeingFwhmGeom/pixscale/2.355')

In [ ]:
psfSigmaEff_opsim = opsim['seeingFwhmEff']/2.355/PIXSCALE
plot_comparison(opsim['psfSigma'], psfSigmaEff_opsim, opsim['band'], 
                xlabel='psfSigma', ylabel='seeingFwhmGeom/pixscale/2.355')

In [ ]:
# If we don't have psfArea, we can approximate it using psfSigma and the relations in SMTN-002 
# and assuming that psfSigma ~ FWHM_geom (which is not a great approximation)
psfFwhm = 2.355 * opsim['psfSigma'] # psfSigma ~ FWHM_geom
psfFwhmEff = (psfFwhm - 0.052/PIXSCALE)/ (0.822)
psfSigmaArea = 2.266 * psfFwhmEff**2

plot_comparison(opsim['psfArea'], psfSigmaArea, opsim['band'], 
                xlabel='psfArea', ylabel='psfSigmaArea')

In [ ]:
# This can be brought into closer agreement if we modify the scaling.
# Modifying the coefficients in the relationship between psfSigma and
# psfFwhmEff can give much better agreement.
psfFwhm = 2.355 * opsim['psfSigma'] # psfSigma ~ FWHM_geom
psfFwhmEff = (psfFwhm - 0.0/PIXSCALE)/ (0.90)
psfSigmaArea = 2.226 * psfFwhmEff**2

plot_comparison(opsim['psfArea'], psfSigmaArea, opsim['band'], 
                xlabel='psfArea', ylabel='psfSigmaArea')